# Getting Started with Vantage: Collection Management

Welcome to the Collection Management part of our [Getting Started with Vantage](https://github.com/VantageDiscovery/vantage-tutorials/tree/main/examples/sdk/python/notebooks/getting_started) series.

A Collection is the fundamental object in the Vantage platform that enables you to organize, manage, and search your data sets within the platform.

Your data records, called documents, are ingested into a collection. Your search queries run against a collection. We currently support text data in collections, but we will soon support other types of data as well.

When creating a collection, you give it an ID, a name, and specify some parameters for the AI model that will be used to embed your collection data.

You can create many collections in your account to separate your different data sets that you want to search against.

Let's see how we can leverage these functionalities using our SDK!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/VantageDiscovery/vantage-tutorials/blob/main/examples/sdk/python/notebooks/getting_started/management_api/collection_management.ipynb)

### ✅ Installation

The first step involves installing the [Vantage](https://pypi.org/project/vantage-sdk/) package.

In [ ]:
! pip install vantage-sdk -qU

As usual, let's import the necessary libraries.

In this example we will need just the `os` library to load our environment variables:

In [ ]:
import os

### ✅ Initialization

In this example, we will authenticate using a Vantage API Key.
For additional details on initializing the Vantage client, refer to the [notebook](../initializing_the_client.ipynb) that covers this topic first.

Please update the following two cells with the appropriate values.

In [ ]:
ACCOUNT_ID = "YOUR_ACCOUNT_ID"
API_HOST = "https://api.dev-a.dev.vantagediscovery.com"

In [ ]:
%env VANTAGE_API_KEY=VANTAGE_API_KEY

In [ ]:
from vantage_sdk import VantageClient

vantage_instance = VantageClient.using_vantage_api_key(
    vantage_api_key=os.environ["VANTAGE_API_KEY"],
    account_id=ACCOUNT_ID,
    api_host=API_HOST,
)

### ✅ Collection Management

The Collection Management API enables the listing of your collections, creation of new ones, updating existing collections, and deleting them.

In the following cells, you will find more details on each of these functionalities.

#### Get All Collections

- Easily access all your collections by calling `list_collections` method.

In [ ]:
collections = vantage_instance.list_collections()
collections

#### Create Collection

There are different types of collections, depending on whether you want Vantage to manage and create embeddings from your data, or if you prefer to provide them manually. The Python SDK provides two types of classes based on this distinction:

UserProvidedEmbeddingCollection and
VantageManagedEmbeddingCollection.
Further, depending on the LLM provider, which is either OpenAI or HuggingFace, Vantage-managed embedding collections are divided into OpenAICollection and HuggingFaceCollection classes. You should instantiate the appropriate one. 

Further details are provided in our [documentation](https://docs.vantagediscovery.com/docs/collections).

**Vantage Managed Embeddings (VME)**

Below, you can see the cells representing the creation of two types of Vantage-managed embeddings collections.

The parameter that represents the actual embedding model (LLM) differs:

- OpenAI collections require the `llm` parameter, where you can provide the OpenAI model name.
- HuggingFace collections use the `external_url` parameter, which represents the endpoint URL of the deployed HF model.

In [ ]:
from vantage_sdk.model.collection import OpenAICollection

openai_collection_id = "vantage-managed-openai"

openai_collection = OpenAICollection(
    collection_id=openai_collection_id,
    embeddings_dimension=1536,
    llm="text-embedding-ada-002",
    llm_secret="YOUR_OPENAI_KEY",
)

created_openai_collection = vantage_instance.create_collection(openai_collection)

created_openai_collection

In [ ]:
from vantage_sdk.model.collection import HuggingFaceCollection

hf_collection_id = "vantage-managed-hf"

hf_collection = HuggingFaceCollection(
    collection_id=hf_collection,
    embeddings_dimension=1536,
    external_url="HF_ENDPOINT_URL",
    external_account_id="YOUR_MODEL_API_KEY_ID",
)

created_hf_collection = vantage_instance.create_collection(hf_collection)

created_hf_collection

**User Provided Embeddings (UPE)**

The cell below describes how you can create a user-provided embeddings collection. It simply requires only the `collection_id` and the `embedding_dimensions` parameters.

> ❗ *The semantic search by text query endpoint will not be avaiable for UPE collections.*

In [ ]:
from vantage_sdk.model.collection import UserProvidedEmbeddingsCollection

upe_collection_id = "user-provided"

upe_collection = UserProvidedEmbeddingsCollection(
    collection_id=upe_collection_id,
    embeddings_dimension=3,
)

created_vme_collection = vantage_instance.create_collection(upe_collection)

created_vme_collection

#### Get One Collection

- Easily access your collection by providing its `collection_id`.

In [ ]:
collection = vantage_instance.get_collection(
    collection_id = upe_collection_id
)

collection

#### Update Collection

- Easily update your collection by providing its `collection_id` along with the specific fields you wish to update.
  - Currently, it is possible to change the `collection_name`.

In [ ]:
updated_collection = vantage_instance.update_collection(
    collection_id = upe_collection_id,
    collection_name = "Updated Collection Name",
)

updated_collection

#### Delete Collection

- Easily delete your collection by providing its `collection_id`.

In [14]:
vantage_instance.delete_collection(
    collection_id = upe_collection_id
)

## 📌 Next Steps

You are now familiar with the Collection Management endpoints! 

You can take a look at other notebooks from our [Getting Started with Vantage](https://github.com/VantageDiscovery/vantage-tutorials/tree/main/examples/sdk/python/notebooks/getting_started) series or continue using Vantage on your own.

If you need some ideas, check our [Tutorials](https://docs.vantagediscovery.com/docs/tutorials), where you can find inspiration and best practices for using Vantage.

Happy discovering! 🔎
